In [1]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.mask import mask
from shapely.geometry import Point, mapping
import geopandas as gpd
from pyproj import Transformer


In [3]:

def create_buffer_500m(lon, lat, crs_raster):
    """
    Tạo buffer 500m quanh điểm OCO-2
    
    Parameters:
    - lon, lat: Tọa độ điểm (độ)
    - crs_raster: CRS của raster ALOS
    
    Returns:
    - buffer geometry trong CRS của raster
    """
    # Tạo point trong WGS84
    point = Point(lon, lat)
    gdf = gpd.GeoDataFrame([1], geometry=[point], crs="EPSG:4326")
    
    # Chuyển sang hệ tọa độ mét (UTM hoặc crs của raster)
    # Nếu ALOS đã có CRS phù hợp, dùng luôn
    # Nếu không, tìm UTM zone phù hợp
    if crs_raster.is_projected:
        gdf_projected = gdf.to_crs(crs_raster)
    else:
        # Tự động tìm UTM zone
        utm_zone = int((lon + 180) / 6) + 1
        hemisphere = 'north' if lat >= 0 else 'south'
        epsg_code = 32600 + utm_zone if hemisphere == 'north' else 32700 + utm_zone
        gdf_projected = gdf.to_crs(f"EPSG:{epsg_code}")
    
    # Tạo buffer 500m
    buffer = gdf_projected.buffer(500)
    
    # Chuyển về CRS của raster
    buffer_raster_crs = buffer.to_crs(crs_raster)
    
    return buffer_raster_crs.geometry.iloc[0]

def check_rice_coverage(lon, lat, raster_path, rice_value=3, threshold=0.5):
    """
    Kiểm tra xem điểm có nằm trong khu vực lúa không
    
    Parameters:
    - lon, lat: Tọa độ điểm OCO-2
    - raster_path: Đường dẫn đến file ALOS
    - rice_value: Giá trị pixel của ruộng lúa (mặc định = 3)
    - threshold: Ngưỡng tỷ lệ pixel lúa (mặc định = 0.5 = 50%)
    
    Returns:
    - True nếu >= threshold là pixel lúa, False nếu không
    """
    try:
        with rasterio.open(raster_path) as src:
            # Tạo buffer 500m
            buffer_geom = create_buffer_500m(lon, lat, src.crs)
            
            # Crop raster theo buffer
            out_image, out_transform = mask(src, [mapping(buffer_geom)], crop=True, nodata=src.nodata)
            
            # Lấy dữ liệu từ band đầu tiên
            data = out_image[0]
            
            # Lọc bỏ NoData
            valid_mask = data != src.nodata
            if src.nodata is None:
                # Nếu không có nodata được định nghĩa, coi tất cả là valid
                valid_data = data.flatten()
            else:
                valid_data = data[valid_mask]
            
            if len(valid_data) == 0:
                return False
            
            # Đếm pixel lúa
            rice_pixels = np.sum(valid_data == rice_value)
            total_pixels = len(valid_data)
            
            # Tính tỷ lệ
            rice_ratio = rice_pixels / total_pixels
            
            return rice_ratio >= threshold
            
    except Exception as e:
        print(f"Lỗi xử lý điểm ({lon}, {lat}): {e}")
        return False

def filter_oco2_rice_areas(oco2_df, raster_path, lon_col='longitude', lat_col='latitude', 
                           rice_value=3, threshold=0.5):
    """
    Lọc toàn bộ dataset OCO-2 để giữ lại các điểm trong khu vực lúa
    
    Parameters:
    - oco2_df: DataFrame chứa dữ liệu OCO-2
    - raster_path: Đường dẫn đến file ALOS
    - lon_col, lat_col: Tên cột chứa kinh độ và vĩ độ
    - rice_value: Giá trị pixel của ruộng lúa
    - threshold: Ngưỡng tỷ lệ pixel lúa
    
    Returns:
    - DataFrame đã lọc và danh sách mask boolean
    """
    print(f"Đang xử lý {len(oco2_df)} điểm OCO-2...")
    
    rice_mask = []
    for idx, row in oco2_df.iterrows():
        lon = row[lon_col]
        lat = row[lat_col]
        
        is_rice = check_rice_coverage(lon, lat, raster_path, rice_value, threshold)
        rice_mask.append(is_rice)
        
        # Hiển thị tiến trình
        if (idx + 1) % 100 == 0:
            print(f"Đã xử lý {idx + 1}/{len(oco2_df)} điểm")
    
    # Lọc dữ liệu
    oco2_rice = oco2_df[rice_mask].copy()
    
    print(f"\nKết quả:")
    print(f"- Tổng số điểm: {len(oco2_df)}")
    print(f"- Điểm trong khu vực lúa: {len(oco2_rice)} ({len(oco2_rice)/len(oco2_df)*100:.2f}%)")
    
    return oco2_rice, rice_mask

# ========== SỬ DỤNG ==========

# 1. Đọc dữ liệu OCO-2
oco2_df = pd.read_csv(r"E:\DownloadData\co2_nasa\data\csv\2023\oco2\oco2_vn_qual0_2023.csv")  # Thay đổi đường dẫn file

# 2. Đường dẫn đến file ALOS đã gộp
alos_path = r'E:\RanhGioi\LandCoverMap\\landcover_vn_250m.tif'  # Thay đổi đường dẫn file

# 3. Lọc các điểm trong khu vực lúa
oco2_rice, rice_mask = filter_oco2_rice_areas(
    oco2_df, 
    alos_path,
    lon_col='longitude',  # Tên cột kinh độ trong DataFrame
    lat_col='latitude',   # Tên cột vĩ độ trong DataFrame
    rice_value=3,         # Giá trị pixel ruộng lúa
    threshold=0.1        # 50% là ngưỡng
)

# 4. Lưu kết quả
oco2_rice.to_csv(r"E:\DownloadData\co2_nasa\data\csv\2023\oco2\oco2_rice_2023.csv", index=False)
print("\nĐã lưu kết quả vào file: oco2_rice_filtered.csv")

# 5. (Tùy chọn) Thêm cột đánh dấu vào DataFrame gốc
oco2_df['is_rice_area'] = rice_mask
oco2_df.to_csv(r"E:\DownloadData\co2_nasa\data\csv\2023\oco2\oco2_rice_flag_2023.csv", index=False)

Đang xử lý 4726 điểm OCO-2...
Đã xử lý 100/4726 điểm
Đã xử lý 200/4726 điểm
Đã xử lý 300/4726 điểm
Đã xử lý 400/4726 điểm
Đã xử lý 500/4726 điểm
Đã xử lý 600/4726 điểm
Đã xử lý 700/4726 điểm
Đã xử lý 800/4726 điểm
Đã xử lý 900/4726 điểm
Đã xử lý 1000/4726 điểm
Đã xử lý 1100/4726 điểm
Đã xử lý 1200/4726 điểm
Đã xử lý 1300/4726 điểm
Đã xử lý 1400/4726 điểm
Đã xử lý 1500/4726 điểm
Đã xử lý 1600/4726 điểm
Đã xử lý 1700/4726 điểm
Đã xử lý 1800/4726 điểm
Đã xử lý 1900/4726 điểm
Đã xử lý 2000/4726 điểm
Đã xử lý 2100/4726 điểm
Đã xử lý 2200/4726 điểm
Đã xử lý 2300/4726 điểm
Đã xử lý 2400/4726 điểm
Đã xử lý 2500/4726 điểm
Đã xử lý 2600/4726 điểm
Đã xử lý 2700/4726 điểm
Đã xử lý 2800/4726 điểm
Đã xử lý 2900/4726 điểm
Đã xử lý 3000/4726 điểm
Đã xử lý 3100/4726 điểm
Đã xử lý 3200/4726 điểm
Đã xử lý 3300/4726 điểm
Đã xử lý 3400/4726 điểm
Đã xử lý 3500/4726 điểm
Đã xử lý 3600/4726 điểm
Đã xử lý 3700/4726 điểm
Đã xử lý 3800/4726 điểm
Đã xử lý 3900/4726 điểm
Đã xử lý 4000/4726 điểm
Đã xử lý 4100/4726 